In [96]:
import numpy as np
import pandas as pd


In [97]:
df=pd.read_csv("diabetes.csv")

In [98]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [99]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [100]:
X=df.iloc[:,:-1].values  # all column except last
y=df.iloc[:,-1].values   # last columns

In [101]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()


In [102]:
X=scaler.fit_transform(X)

In [103]:
X.shape

(768, 8)

In [104]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [146]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [106]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='relu'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])



c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [107]:
model.fit(X_train,y_train,batch_size=10,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7085 - loss: 1.8822 - val_accuracy: 0.7532 - val_loss: 1.4252
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7150 - loss: 1.5669 - val_accuracy: 0.7532 - val_loss: 1.3182
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7134 - loss: 1.3571 - val_accuracy: 0.7403 - val_loss: 1.1585
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7182 - loss: 1.1404 - val_accuracy: 0.7338 - val_loss: 1.1500
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7101 - loss: 1.0187 - val_accuracy: 0.7338 - val_loss: 1.0428
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7199 - loss: 0.9107 - val_accuracy: 0.7403 - val_loss: 1.0371
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7280 - loss: 0.8600 - val_accuracy: 0.7597 - val_loss: 0.9606
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7264 - loss: 0.8247 - val_accuracy: 0.7597 - v

In [108]:
# how to select the appropriate optimizer
# No of nodes in a layer
# how to select no. of layers
# All in all one model

In [109]:
pip install -U keras-tuner

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: keras-tuner in c:\users\rosha\anaconda3\lib\site-packages (1.4.8)



In [110]:
import kerastuner as kt

In [111]:
def build_model(hp):
    model=Sequential()

    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='relu'))

    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmprop','adadelta'])

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])


    return model

    

In [112]:
# tnner object baneyege

tuner=kt.RandomSearch(build_model,objective='val_accuracy',
max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [113]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [114]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [115]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [116]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [117]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6938 - loss: 2.2507 - val_accuracy: 0.7208 - val_loss: 1.7366
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7003 - loss: 2.0070 - val_accuracy: 0.7208 - val_loss: 1.6825
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7117 - loss: 1.5923 - val_accuracy: 0.7078 - val_loss: 1.5884
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6938 - loss: 1.4952 - val_accuracy: 0.6883 - val_loss: 1.3154
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7003 - loss: 1.4027 - val_accuracy: 0.7143 - val_loss: 1.1142
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7036 - loss: 1.1357 - val_accuracy: 0.7143 - val_loss: 1.0093
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7134 - loss: 0.9761 - val_accuracy: 0.6948 - val_loss: 0.9331
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7215 - loss: 0.8944 - val_accuracy: 0.68

In [118]:
def build_model(hp):
    model=Sequential()

    units=hp.Int('units',min_value=8,max_value=128,step=8)

    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='relu'))

    # optimizer=hp.Choice('optimizer',values=['adam','sgd','rmprop','adadelta'])

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])


    return model

    

In [119]:
# tnner object baneyege

tuner=kt.RandomSearch(build_model,objective='val_accuracy',
max_trials=5
,directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [120]:
tuner.search(X_train,y_train,epochs=5,
 validation_data=(X_test,y_test))

In [121]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [122]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [123]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7720 - loss: 0.6700  
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7834 - loss: 0.5997 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7883 - loss: 0.5654 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7834 - loss: 0.5659 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7915 - loss: 0.5328 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7915 - loss: 0.5716 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7834 - loss: 0.5553 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7883 - loss: 0.5388 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7899 - loss: 0.5390 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7834 - loss: 0.5352 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7899 - loss: 0.5331 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

In [127]:
# no of layer

def build_model(hp):
    model=Sequential()
# input layer
    model.add(Dense(72,activation='relu',input_dim=8))
#hidden layer

    for i in range(hp.Int('num_layer',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy'])


    return model




In [128]:
# tnner object baneyege

tuner=kt.RandomSearch(build_model,
objective='val_accuracy',
max_trials=5,
directory='mydir',
project_name='ram')

c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [129]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 15s


In [131]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 2}

In [132]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [134]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7866 - loss: 0.4602 - val_accuracy: 0.8182 - val_loss: 0.4594
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7818 - loss: 0.4431 - val_accuracy: 0.7987 - val_loss: 0.4706
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7850 - loss: 0.4392 - val_accuracy: 0.7662 - val_loss: 0.4667
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7866 - loss: 0.4344 - val_accuracy: 0.7597 - val_loss: 0.4630
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7834 - loss: 0.4289 - val_accuracy: 0.7922 - val_loss: 0.4569
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.4233 - val_accuracy: 0.7857 - val_loss: 0.4661
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8029 - loss: 0.4188 - val_accuracy: 0.8052 - val_loss: 0.4618
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7980 - loss: 0.4123 - val_accuracy: 0.78

In [147]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(hp):

    model = Sequential()

    # Hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

        model.add(Dense(
            units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),

            activation=hp.Choice(
                'activation_' + str(i),
                values=['relu', 'tanh', 'sigmoid']
            )
        ))

    # Output layer (must be 1 for binary classification)
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer=hp.Choice(
            'optimizer',
            values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [148]:
# tnner object baneyege

tuner=kt.RandomSearch(build_model,
objective='val_accuracy',
max_trials=3,
directory='mydir',
project_name='final')

Reloading Tuner from mydir\final\tuner0.json


In [149]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [150]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units_0': 64,
 'activation_0': 'sigmoid',
 'optimizer': 'nadam',
 'units_1': 16,
 'activation_1': 'tanh',
 'units_2': 112,
 'activation_2': 'tanh',
 'units_3': 40,
 'activation_3': 'relu',
 'units_4': 72,
 'activation_4': 'tanh',
 'units_5': 64,
 'activation_5': 'tanh',
 'units_6': 24,
 'activation_6': 'relu',
 'units_7': 72,
 'activation_7': 'tanh',
 'units_8': 80,
 'activation_8': 'tanh'}

In [151]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\rosha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 27 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [152]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7687 - loss: 0.4823 - val_accuracy: 0.7922 - val_loss: 0.4718
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7671 - loss: 0.4780 - val_accuracy: 0.7857 - val_loss: 0.4728
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7671 - loss: 0.4784 - val_accuracy: 0.7727 - val_loss: 0.4931
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7622 - loss: 0.4831 - val_accuracy: 0.7662 - val_loss: 0.4886
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4704 - val_accuracy: 0.7857 - val_loss: 0.4714
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7769 - loss: 0.4727 - val_accuracy: 0.7857 - val_loss: 0.4705
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7704 - loss: 0.4723 - val_accuracy: 0.7857 - val_loss: 0.4699
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7687 - loss: 0.4728 - val_accuracy: 0.77